# 🧠 Black-Scholes Option Pricing Model – Full Notebook

This notebook implements the Black-Scholes formula from scratch, including:
- Formula for call & put options
- Greeks
- Dividend-adjusted pricing
- Monte Carlo simulation
- Implied volatility estimation
- Visualization of pricing surface

In [13]:
# 📌 Import Required Libraries
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import matplotlib.pyplot as plt

In [14]:
# 🧮 Black-Scholes Pricing Function
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2)* T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return price

In [15]:
# 🧪 Sample Inputs
S = 100
K = 100
T = 1
r = 0.05
sigma = 0.2

print("Call Option:", black_scholes(S, K, T, r, sigma, 'call'))
print("Put Option:", black_scholes(S, K, T, r, sigma, 'put'))

Call Option: 10.450583572185565
Put Option: 5.573526022256971


In [16]:
# 📈 Greeks Calculation
def black_scholes_greeks(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    delta = norm.cdf(d1) if option_type == 'call' else norm.cdf(d1) - 1
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega  = S * norm.pdf(d1) * np.sqrt(T)
    theta_call = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2))
    theta_put = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2))
    theta = theta_call if option_type == 'call' else theta_put
    rho_call = K * T * np.exp(-r * T) * norm.cdf(d2)
    rho_put = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    rho = rho_call if option_type == 'call' else rho_put

    return {
        "Delta": delta,
        "Gamma": gamma,
        "Vega": vega / 100,
        "Theta": theta / 365,
        "Rho": rho / 100
    }

In [17]:
# 💸 Black-Scholes with Continuous Dividends
def black_scholes_with_dividends(S, K, T, r, sigma, q=0.0, option_type='call'):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2)* T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r*T) * norm.cdf(-d2) - S * np.exp(-q*T) * norm.cdf(-d1)
    return price

In [18]:
# 🎲 Monte Carlo Simulation
def monte_carlo_option_price(S, K, T, r, sigma, simulations=10000, option_type='call'):
    np.random.seed(42)
    Z = np.random.standard_normal(simulations)
    ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    if option_type == 'call':
        payoff = np.maximum(ST - K, 0)
    else:
        payoff = np.maximum(K - ST, 0)

    return np.exp(-r * T) * np.mean(payoff)

In [19]:
# 🔍 Implied Volatility
def implied_volatility(target_price, S, K, T, r, option_type='call'):
    def objective(sigma):
        return black_scholes(S, K, T, r, sigma, option_type) - target_price
    return brentq(objective, 1e-5, 3.0)

In [20]:
greeks = black_scholes_greeks(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type='call')
for g, val in greeks.items():
    print(f"{g}: {val:.4f}")


Delta: 0.6368
Gamma: 0.0188
Vega: 0.3752
Theta: -0.0176
Rho: 0.5323
